In [1]:
# import necessary packages and utils

import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models, transforms, datasets

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset

import PIL.Image as Image
from tqdm import tqdm
import pickle

from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error as mse
from sklearn import linear_model
from sklearn.model_selection import StratifiedShuffleSplit

from models.vgg16 import *

In [2]:
# config   
num_epochs = 1
num_classes = 2
batch_size = 16
learning_rate = 1e-5
# model_name = vgg16_bn(pretrained=True) # baseline model

In [3]:
# load cvs data

data_dir = "../data/petfinder-pawpularity-score/"

img_train_dir = os.path.join(data_dir, 'train')

def return_imgfilepath(name, folder=img_train_dir):
    path = os.path.join(folder, f'{name}.jpg')
    return path

train_file_path = os.path.join(data_dir, 'train.csv')
train_df = pd.read_csv(train_file_path)

# im = Image.open(train_df['file_path'][0])

# set image filepath
train_df['file_path'] = train_df['Id'].apply(lambda x: return_imgfilepath(x))
train_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,file_path
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../data/petfinder-pawpularity-score/train\0007...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../data/petfinder-pawpularity-score/train\0009...
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../data/petfinder-pawpularity-score/train\0013...
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../data/petfinder-pawpularity-score/train\0018...
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../data/petfinder-pawpularity-score/train\001d...


In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
device

device(type='cuda')

In [5]:
class PetDataset(Dataset):
    def __init__(self, image_filepaths, targets, transform=None):
        self.image_filepaths = image_filepaths
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.image_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.image_filepaths[idx]
        with open(image_filepath, 'rb') as f:
            image = Image.open(f)
            image_rgb = image.convert('RGB')
        # image = np.array(image_rgb) / 255 # convert to 0-1

        if self.transform is not None:
            image = self.transform(image)
        
        target = self.targets[idx]

        image = torch.tensor(image)
        target = torch.tensor(target)
        
        return image, target

In [6]:
# get data
# train_loader, val_loader, test_loader = load_data(batch_size, use_subset=True)

def load_data(batch_size, use_subset=True):
    """
    return the train dataloader
    """
    
    images = np.array(train_df['file_path'])
    targets = np.array(train_df['Pawpularity'])
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
    ])
    
    train_dataset = PetDataset(image_filepaths=images, targets=targets, transform=transform)
    
    indices_train = list(range(700))
    
    train_subset = Subset(train_dataset, indices_train)

    # data loader
    train_loader = DataLoader(dataset=train_subset if use_subset else train_dataset, 
                                batch_size=batch_size,
                                shuffle=True)
    
    return train_loader

train_loader = load_data(batch_size, use_subset=False)

In [7]:
# init model

class ConvNet_v1(nn.Module):
    """
    Simple two-layer CNN with sequential container
    """
    def __init__(self):
        super(ConvNet_v1, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(387200, 128),
            nn.ReLU(),
            nn.Linear(128, 1) 
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out).squeeze()
        return out

In [8]:
# init model

def initialize_model(model, learning_rate, num_classes):
    """
    initialize the model
    define loss function and optimizer and move data to gpu if available
    
    return:
        model, loss function(criterion), optimizer
    """
    
    model = model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    return model, criterion, optimizer

model_name = ConvNet_v1()

model, criterion, optimizer = initialize_model(model_name, learning_rate, num_classes)

In [9]:
experiment_name = "simple_run"

In [10]:
# train

def calc_rmse(y_pred, y_true):
    return np.sqrt(((y_pred - y_true) ** 2).mean())


def train(train_loader, model, criterion, optimizer, num_epochs):
    """
    Move data to GPU memory and train for specified number of epochs
    Also plot the loss function and save it in `Figures/`
    Trained model is saved as `cnn.ckpt`
    """
    for epoch in range(num_epochs): # repeat the entire training `num_epochs` times
        # for each training sample
        loss_hist = []
        step_hist = []
        for i, (images, label) in (enumerate(train_loader)):
            
            model.train()
            train_pred = list()
            train_true = list()
            
            # move to gpu if available
            images = images.to(device).float()
            label = label.to(device).float()
                
            # forward pass
            out = model(images)
            loss = torch.sqrt(criterion(out, label))

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_true += label.cpu().detach().numpy().tolist()
            train_pred += out.cpu().detach().numpy().tolist()

            train_rmse = calc_rmse(np.array(train_pred), np.array(train_true))
            
            if i // 500 == 0:
                print('Train rmse: {}'.format(train_rmse))
        
    torch.save(model.state_dict(), experiment_name+'.ckpt')

In [11]:
train(train_loader, model, criterion, optimizer, num_epochs)

<ipython-input-5-8e7a1010962a>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image)


Train rmse: 47.15507356055132
Train rmse: 45.10403412699402
Train rmse: 45.736826632377024
Train rmse: 41.60271222155497
Train rmse: 40.519040056564144
Train rmse: 36.063611517080375
Train rmse: 39.82541384496927
Train rmse: 35.6499764993893
Train rmse: 35.52078764687036
Train rmse: 35.46592146958015
Train rmse: 43.788651054303635
Train rmse: 38.548666216178745
Train rmse: 35.55924622520833
Train rmse: 41.21558031693333
Train rmse: 44.0316052784688
Train rmse: 37.168011031924785
Train rmse: 38.126709052825845
Train rmse: 44.20220169419213
Train rmse: 29.13291394759467
Train rmse: 22.35828081740603
Train rmse: 35.61235746332799
Train rmse: 31.831863473174685
Train rmse: 24.157567965495826
Train rmse: 25.11416013860125
Train rmse: 25.02456306188615
Train rmse: 25.293954970941396
Train rmse: 29.77659174602438
Train rmse: 26.375269433723737
Train rmse: 31.150962533438413
Train rmse: 35.826632892601815
Train rmse: 33.901797759564154
Train rmse: 33.60579560373354
Train rmse: 32.6029628974267

Train rmse: 16.543997343490943
Train rmse: 24.962350185291047
Train rmse: 11.607221059232357
Train rmse: 21.940751706519105
Train rmse: 25.340127260722316
Train rmse: 16.353303372282888
Train rmse: 13.697772371348126
Train rmse: 12.776063903060736
Train rmse: 17.53440923783763
Train rmse: 23.94543192385702
Train rmse: 19.215981515501678
Train rmse: 21.207961821159504
Train rmse: 17.455767435130838
Train rmse: 20.73267795032421
Train rmse: 16.913931365990585
Train rmse: 15.816491050191745
Train rmse: 25.95046152802724
Train rmse: 25.31173778635381
Train rmse: 18.522510193474435
Train rmse: 26.992309512459624
Train rmse: 24.481124611227308
Train rmse: 18.194520795549664
Train rmse: 22.575896170386464
Train rmse: 21.620373919878094
Train rmse: 16.89439685323002
Train rmse: 20.43565269937307
Train rmse: 19.257766395312945
Train rmse: 13.604590916336068
Train rmse: 27.75980612145873
Train rmse: 16.41601135792106
Train rmse: 19.360140547726115
Train rmse: 24.892725355510567
Train rmse: 24.99

In [26]:
data_dir = "../data/petfinder-pawpularity-score/"

img_test_dir = os.path.join(data_dir, 'test')

def return_imgfilepath(name, folder=img_test_dir):
    path = os.path.join(folder, f'{name}.jpg')
    return path

test_file_path = os.path.join(data_dir, 'test.csv')
test_df = pd.read_csv(test_file_path)

# set image filepath
test_df['file_path'] = test_df['Id'].apply(lambda x: return_imgfilepath(x))
test_df

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,file_path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../data/petfinder-pawpularity-score/test\4128b...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../data/petfinder-pawpularity-score/test\43a22...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,../data/petfinder-pawpularity-score/test\4e429...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,../data/petfinder-pawpularity-score/test\80bc3...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,../data/petfinder-pawpularity-score/test\8f498...
5,b03f7041962238a7c9d6537e22f9b017,0,0,1,1,1,1,1,1,1,0,1,0,../data/petfinder-pawpularity-score/test\b03f7...
6,c978013571258ed6d4637f6e8cc9d6a3,1,0,0,0,1,1,0,1,0,1,1,1,../data/petfinder-pawpularity-score/test\c9780...
7,e0de453c1bffc20c22b072b34b54e50f,1,0,1,0,0,0,0,0,1,0,0,1,../data/petfinder-pawpularity-score/test\e0de4...


In [27]:
def load_data_test(batch_size, use_subset=True):
    """
    return the train dataloader
    """
    
    images = np.array(test_df['file_path'])
    targets = np.zeros_like(images)
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
    ])
    
    test_dataset = PetDataset(image_filepaths=images, targets=targets, transform=transform)

    # data loader
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
    
    return test_loader

test_loader = load_data_test(batch_size, use_subset=False)

In [28]:
model

ConvNet_v1(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Flatten(start_dim=1, end_dim=-1)
  )
  (layer2): Sequential(
    (0): Linear(in_features=387200, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [29]:
# eval

# iterate over test data
print('Making predictions...')
test_pred = list()
for test_images, test_labels in test_loader:
    test_images = test_images.to(device).float()
    
    # forward
    out = model(test_images)
    test_pred += out.cpu().detach().numpy().tolist()

# write to file
print('Write to file...')
output = pd.DataFrame({"Id": test_df['Id'], "Pawpularity": test_pred})
output.to_csv('submission.csv', index = False)

Making predictions...
Write to file...


<ipython-input-5-8e7a1010962a>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image)


In [30]:
# check output

output_df = pd.read_csv('submission.csv')
output_df

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,41.356541
1,43a2262d7738e3d420d453815151079e,41.437817
2,4e429cead1848a298432a0acad014c9d,41.401272
3,80bc3ccafcc51b66303c2c263aa38486,41.459965
4,8f49844c382931444e68dffbe20228f4,41.504787
5,b03f7041962238a7c9d6537e22f9b017,41.427372
6,c978013571258ed6d4637f6e8cc9d6a3,41.409576
7,e0de453c1bffc20c22b072b34b54e50f,41.474960
